https://www.kaggle.com/datasets/hojjatk/mnist-dataset

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


data = pd.read_csv(r'../../../2. MNIST_dataset\train.csv')
data.sample(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
18481,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37149,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31034,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
data.iloc[0]

label       1
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Name: 0, Length: 785, dtype: int64

In [ ]:
data.shape

(42000, 785)

In [ ]:
data = np.array(data)
data

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [7, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0]])

In [ ]:
m, n = data.shape
m, n

(42000, 785)

In [ ]:
np.random.shuffle(data)
data

array([[5, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       ...,
       [3, 0, 0, ..., 0, 0, 0],
       [8, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0]])

In [ ]:
# picking only 1000 examples for training
data_dev = data[:1000, :]

# transposing data, so that features lie down horizontally, as we visualize the input of a NN architecture.
data_dev = data_dev.T
data_dev.shape

(785, 1000)

In [ ]:
data_dev

array([[5, 9, 2, ..., 5, 0, 3],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
y_dev = data_dev[0]
x_dev = data_dev[1:]

In [ ]:
data_train = data[10000:].T
y_train = data_train[0]
x_train = data_train[1:]

In [ ]:
def init_params():
    h1_w1 = np.random.randn(10, 784) - 0.5
    h1_b1 = np.random.randn(10, 1) - 0.5

    h2_w2 = np.random.randn(10, 10) - 0.5
    h2_b2 = np.random.randn(10, 1) - 0.5

    return h1_w1, h1_b1, h2_w2, h2_b2

def relu(z):
    return np.maximum(0, z)

def softmax(z):
    return np.exp(z) / np.sum(np.exp(z))

def forward_prop(h1_w1, h1_b1, h2_w2, h2_b2):
    z1 = h1_w1.dot(x_train) + h1_b1
    a1 = relu(z1)
    
    z2 = h2_w2.dot(a1) + h2_b2
    a2 = softmax(z2)

    return z1, a1, z2, a2

def one_hot():
    

def back_prop(z1, a1, z2, a2):
